In [297]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [502]:
# NOTES:
# - use numpy instead of pandas for speed

from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
df = pd.DataFrame(iris.data)

df.columns = iris.feature_names
# X = df[df.columns]
X = X = df[df.columns].values

df["species"] = iris.target
# y = df["species"]
y = df["species"].values
# print(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [848]:
class Node:
    def __init__(self, feature=None, index=None, n_samples=0, threshold=None):
        self.feature = feature
        self.feature_index = index
        self.threshold = threshold 
        self.n_samples = n_samples
        self.predicted_class = 1
        self.left = None
        self.right = None
    
    def debug(self, feature_names, class_names, show_details):
        """Print an ASCII visualization of the tree."""
        lines, _, _, _ = self._debug_aux(
            feature_names, class_names, show_details, root=True
        )
        for line in lines:
            print(line)
    
    def _debug_aux(self, feature_names, class_names, show_details, root=False):
        # See https://stackoverflow.com/a/54074933/1143396 for similar code.
        is_leaf = not self.right
        if is_leaf:
            lines = [class_names[self.predicted_class]]
        else:
            lines = [
                "{} < {:.2f}".format(feature_names[self.feature_index], self.threshold)
            ]
#         if show_details:
#             lines += [
#                 "gini = {:.2f}".format(self.gini),
#                 "samples = {}".format(self.num_samples),
#                 str(self.num_samples_per_class),
#             ]
        width = max(len(line) for line in lines)
        height = len(lines)
        if is_leaf:
            lines = ["║ {:^{width}} ║".format(line, width=width) for line in lines]
            lines.insert(0, "╔" + "═" * (width + 2) + "╗")
            lines.append("╚" + "═" * (width + 2) + "╝")
        else:
            lines = ["│ {:^{width}} │".format(line, width=width) for line in lines]
            lines.insert(0, "┌" + "─" * (width + 2) + "┐")
            lines.append("└" + "─" * (width + 2) + "┘")
            lines[-2] = "┤" + lines[-2][1:-1] + "├"
        width += 4  # for padding

        if is_leaf:
            middle = width // 2
            lines[0] = lines[0][:middle] + "╧" + lines[0][middle + 1 :]
            return lines, width, height, middle

        # If not a leaf, must have two children.
        left, n, p, x = self.left._debug_aux(feature_names, class_names, show_details)
        right, m, q, y = self.right._debug_aux(feature_names, class_names, show_details)
        top_lines = [n * " " + line + m * " " for line in lines[:-2]]
        # fmt: off
        middle_line = x * " " + "┌" + (n - x - 1) * "─" + lines[-2] + y * "─" + "┐" + (m - y - 1) * " "
        bottom_line = x * " " + "│" + (n - x - 1) * " " + lines[-1] + y * " " + "│" + (m - y - 1) * " "
        # fmt: on
        if p < q:
            left += [n * " "] * (q - p)
        elif q < p:
            right += [m * " "] * (p - q)
        zipped_lines = zip(left, right)
        lines = (
            top_lines
            + [middle_line, bottom_line]
            + [a + width * " " + b for a, b in zipped_lines]
        )
        middle = n + width // 2
        if not root:
            lines[0] = lines[0][:middle] + "┴" + lines[0][middle + 1 :]
        return lines, n + m + width, max(p, q) + 2 + len(top_lines), middle

In [849]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=3):
        self.tree = None
        self.features = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)']
        self.classes = ["S1", "S2", "S3"]
        self.max_depth = max_depth
        
    def _get_feature_gini(self, feature, threshold, X_sample, y_sample):
        """
        Determines the probability of classifying a point incorrectly.
        Use this method if the target values are classifications.
        """
        
        left = { _class:0 for _class in set(y_sample) }
        right = { _class:0 for _class in set(y_sample) }
        
        for i in range(len(X_sample[:, feature])):
            point = X_sample[i, feature]
            point_class = y_sample[i]
            # assumes integers are continuous
            if is_continuous(point):
                if point <= threshold:
                    left[point_class] += 1
                else:
                    right[point_class] += 1
            else:
                if point == threshold:
                    left[point_class] += 1
                else:
                    right[point_class] += 1
        
        total_left = sum(left.values())
        total_right = sum(right.values())
        
        if total_left != 0:
            gini_left = 1 - sum((class_count/total_left) ** 2 for class_count in left.values())
        else:
            gini_left = 0
        
        if total_right != 0:
            gini_right = 1 - sum((class_count/total_right) ** 2 for class_count in right.values())
        else:
            gini_right = 0
                
        overall_gini = (total_left/len(X_sample) * gini_left) + (total_right/len(X_sample) * gini_right)

        return gini_left, gini_right, overall_gini
    
    def _get_best_split(self, X_sample, y_sample): # return best feature and it's threshold
#         if len(y) <= 1:
#             return None, None
        feature_min_gini = [1] * len(self.features)
        best_feats_info = [
            {
                "index": None,
                "name": None,
                "gini":1, 
                "threshold":None, 
                "left_gini":False, 
                "right_gini":False
            }
        for _ in range(len(self.features))] 
    
        for feat_i, feat in enumerate(self.features):
            values = sorted(np.unique(X_sample[:, feat_i]))
            best_feats_info[feat_i]["index"] = feat_i
            best_feats_info[feat_i]["name"] = feat
            for i in range(len(values) - 1):
                threshold = (values[i] + values[i + 1]) / 2
                left_gini, right_gini, overall_gini = self._get_feature_gini(feat_i, threshold, X_sample, y_sample)
                if overall_gini < best_feats_info[feat_i]["gini"]:
                    best_feats_info[feat_i]["gini"] = overall_gini
                    best_feats_info[feat_i]["threshold"] = threshold
                    best_feats_info[feat_i]["left_gini"] = True if left_gini == 0 else False
                    best_feats_info[feat_i]["right_gini"] = True if right_gini == 0 else False
        feat = np.argmin([feat["gini"] for feat in best_feats_info])
        return best_feats_info[feat]
    
    def _build_tree(self, X_train, y_train, depth=0):
        new_node = Node()
        
        if depth < self.max_depth:
            f = self._get_best_split(X_train, y_train)
            feat_index, feat_name, gini, threshold, left_is_leaf, right_is_leaf = f["index"], f["name"], f["gini"], f["threshold"], f["left_gini"], f["right_gini"]
            
#             print(feat_index, threshold)

            if feat_index is not None:
                left_X, right_X, left_y, right_y = [], [], [], []
                for X_row, y in zip(X_train, y_train):
                    if X_row[feat_index] <= threshold:
                        left_X.append(X_row)
                        left_y.append(y)
                    else:
                        right_X.append(X_row)
                        right_y.append(y)
                new_node.feature = feat_index
                new_node.threshold = threshold
                
                if not left_is_leaf:
#                     print("LEFT")
                    new_node.left = self._build_tree(np.array(left_X), np.array(left_y), depth + 1)
                else:
                    print(feat_index)
                    new_node.left = Node(feature=self.features[feat_index], index=feat_index, threshold=threshold, n_samples=len(left_X))
                    return new_node.left
                    
                if not right_is_leaf:
#                     print("RIGHT")
                    new_node.right = self._build_tree(np.array(right_X), np.array(right_y), depth + 1)
                else:
                    new_node.right = Node(feature=self.features[feat_index], index=feat_index, threshold=threshold, n_samples=len(right_X))
                    return new_node.right
            else:
                new_node.feature = feat_index
                new_node.threshold = threshold
        return new_node

    def _debug(self, feature_names, class_names, show_details=True):
        """Print ASCII visualization of decision tree."""
        self.tree.debug(feature_names, class_names, show_details)
    
    def _confusion_matrix(self, y_pred, y_test):
        pass

    def fit(self, X_train, y_train):
#         self.features = X_train.columns
        self.tree = self._build_tree(X_train, y_train)
#         self._debug(X_train.columns)

    def predict(self, x_test):
        pass

    def visualize(self):
        self._debug(self.features, self.classes)

    def evaluate(self):
        pass

    def compare_with_sklearn(self):
        pass

In [850]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt.visualize()
# dt._get_feature_gini(3, 0.8, X_train, y_train)
# dt._get_best_split(X_train, y_train)

2
╔══╧═╗
║ S2 ║
╚════╝


In [851]:
# for y in y_train:
#     print(y)
# for X_row in X_train.values:
#     print(X_row)
# print(X_train.columns)
# print(X_train.values[0])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import pydotplus

clf = DecisionTreeClassifier(criterion='gini', max_depth=3)
clf.fit(X_train, y_train)
dot_data = export_graphviz(clf, out_file=None, feature_names=['sepal length', 'sepal width', 'petal length', 'petal width'])

graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_png('sklearn_iris_tree.png')

In [42]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'species'],
      dtype='object')

In [441]:
# for row in X_train.values:
#     print(row)

# X_train[:,2]

In [ ]:
# MISC CODE

#     def _mean_squared_error(self, feature):
#         """
#         Use this method if target values are continuous.
#         """
#         X_feature = self.X_train[feature]
#         plt.scatter(X_feature, self.y_train)
#         pass

#         features = self.X_sample.columns
# #         best_gini = 1
# #         best_thres = None
    
# #     def _get_optimal_threshold(self, feature):
# #         feature_values = sorted(self.X_sample[feature].unique())
# #         min_gini = 1
# #         optimal_thres = 0
            
#             best_gini = 1
#             best_thres = None
        
#             for i in range(len(feature_values)-1):
#                 threshold = (feature_values[i] + feature_values[i + 1]) / 2
#                 gini = self._get_overall_gini(feature, threshold)
#                 if gini < min_gini:
#                     min_gini = gini
#                     optimal_thres = threshold
# #         return min_gini, optimal_thres
    
#     def _get_optimal_feature(self):
#         features = self.X_sample.columns
        
#         optimal_feat = np.argmin([self._get_optimal_threshold(feat)[0] for feat in features])
#         return features[optimal_feat]

In [320]:
class Question:
    def __init__(self, feature, threshold):
        self.feature = feature
        self.threshold = threshold
        
    def answer(self, point):
        val = point[self.feature]
        if is_continuous(val): # currently assumes integer values are not categorical
            return val <= self.threshold
        else:
            return val == self.threshold
    
    # Credit: Josh Gordon from Google
    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_continuous(self.threshold):
            condition = ">="
        return f"Is {self.feature} {condition} {self.threshold}?"

In [ ]:
#                 if overall_gini < feature_min_gini[feat_i]:
#                     feature_min_gini[feat_i] = overall_gini
#                 if overall_gini < best_gini:
#                     if left_gini == 0:
#                         left_is_leaf = True
#                     if right_gini == 0:
#                         right_is_leaf = True
#                     best_gini, best_feat_index, best_thres = overall_gini, feat_i, threshold
#         print(feature_min_gini)  
#             print(best_feats_info)
#         for index, feat in enumerate(best_feats_info):
#             print(feat)
#         for feat in (best_feats_info):
#             print(feat)